In [50]:
from sqlalchemy import create_engine, text
import pandas as pd

import credentials

In [56]:
conn_str = 'mysql+mysqlconnector://' + \
	f"{credentials.mysql['username']}:{credentials.mysql['password']}" + \
	'@localhost:3306/Mutis'
engine = create_engine(conn_str)
connection = engine.connect()

In [57]:
ids = pd.read_sql_table("Identifications", connection)

/Users/nelson/anaconda3/envs/py312/lib/python3.12/site-packages/pandas/io/sql.py:1737: SAWarning: Did not recognize type 'polygon' of column 'Shape'
  self.meta.reflect(bind=self.con, only=[table_name], views=True)


In [84]:
noiddate = pd.read_sql(
	"select IdentificationID,DateEnd from Identifications left join Occurrences on Occurrence=OccurrenceID where Date is NULL",
	connection)

In [87]:
ids = ids.merge(noiddate, how='left', on='IdentificationID')

In [95]:
ids.loc[ids.Date.isnull(), 'Date'] = pd.to_datetime(ids.loc[ids.Date.isnull(), 'DateEnd'])

In [ ]:
# Debugging selecting last ided occurrences
#dt = pd.DataFrame.from_dict({
#	'd':['2014-10-16', '2014-10-18', '2014-10-16', '2016-10-16', None, None],
#	'v':[1,1,2,2,2,3],
#	'a':[3,4,4,3,3,4],
#	'i': [1,2,3,4,5,6]})
#dt['d'] = pd.to_datetime(dt.d)
#l = dt.groupby(['v']).d.idxmax()
#onlynas = dt.loc[dt.v.isin(l[l.isnull()].index)].groupby('v').first().reset_index()[dt.columns] # Only NaT dates
#pd.concat([onlynas, dt.loc[l[l.notna()]]])

,d,v,a,i
0,NaT,3,4,6
1,2014-10-18,1,4,2
3,2016-10-16,2,3,4


In [144]:
lastids = ids.groupby('Occurrence').Date.idxmax()

In [154]:
onlynas = ids.loc[
		ids.Occurrence.isin(lastids[lastids.isnull()].index)
	].groupby('Occurrence'
	).first(
	).reset_index(
	)[ids.columns]

In [160]:
id_pool = pd.concat([onlynas, ids.loc[lastids[lastids.notnull()]]]).IdentificationID

In [161]:
tax = pd.read_sql_table("Taxa", connection)

In [162]:
clades = [
	'Polypodiidae',
	'Lycopodiopsida',
	'Ophioglossidae',
	'Equisetidae',
	'Marattiidae'
]

In [163]:
target = []
papa = tax.loc[tax.Name.isin(clades), 'TaxonID'].tolist()

while papa:
	target += papa
	papa = tax.loc[tax.Parent.isin(papa), 'TaxonID'].tolist()
	#print(papa)

In [164]:
tax.loc[tax.TaxonID.isin(target)].head().sort_values('Name')

,TaxonID,Name,AcceptedName,Authority,Author,Protologue,Parent,Comment,CheckPriority,Distribution,TimeStamp
65,66,Acrostichum aureum,None,None,L.,None,4002.0,None,NaN,None,2025-06-18 09:14:24
66,67,Adiantum,None,None,None,None,2934.0,None,NaN,None,2025-06-18 09:14:24
67,68,Adiantum concinnum,None,None,Humb. & Bonpl. ex Willd.,None,67.0,None,NaN,None,2025-06-18 09:14:24
68,69,Adiantum patens,None,None,Willd.,None,67.0,None,NaN,None,2025-06-18 09:14:24
69,70,Adiantum petiolatum,None,None,Desv.,None,67.0,None,NaN,None,2025-06-18 09:14:24


In [177]:
poollist = ', '.join([str(x) for x in id_pool.tolist()])

In [178]:
pd.read_sql(
	"select * from Occurrences left join Identifications on Occurrence=OccurrenceID " \
	+ "left join Locations on LocationID=Location " \
	+ f"where Identifications.IdentificationID IN ({poollist})",
	connection)

,OccurrenceID,DB2015ID,Type,Reference,Collector,CollectorVerbatim,CollectionNumber,CollectionNumberVerbatim,Location,DateInit,...,Longitude,ElevationMin,ElevationMax,Name,Country,Admin01,Admin02,Admin03,Area,TimeStamp
0,1,NaN,Specimen,NaN,None,None,NaN,None,971,1999-02-10,...,NaN,NaN,NaN,"Bogotá, D.C., Bogotá, Cerro de Monserrate, áre...",None,Bogotá,Bogotá,None,None,2025-06-18 09:15:47
1,2,NaN,Specimen,NaN,None,None,NaN,None,971,1999-02-18,...,NaN,NaN,NaN,"Bogotá, D.C., Bogotá, Cerro de Monserrate, áre...",None,Bogotá,Bogotá,None,None,2025-06-18 09:15:47
2,3,NaN,Specimen,NaN,None,None,NaN,None,13981,None,...,-74.010472,3165.0,3165.0,"Bogotá, D.C., Bogotá, Usaquén, cerros de Torca",None,Bogotá,Bogotá,None,None,2025-06-18 09:15:47
3,4,NaN,Specimen,NaN,None,None,NaN,None,965,1999-02-10,...,NaN,NaN,NaN,"Bogotá, D.C., Bogotá, Cerro de Monserrate, áre...",None,Bogotá,Bogotá,None,None,2025-06-18 09:15:47
4,5,NaN,Specimen,NaN,None,None,NaN,None,13371,None,...,-74.033889,3298.0,3298.0,"Bogotá, D.C., Bogotá, Santafé. Páramo de Cruz ...",None,Bogotá,Bogotá,None,None,2025-06-18 09:15:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42839,42840,NaN,Specimen,NaN,None,"Rodríguez, Ángela",2675.0,2675,11629,2019-07-16,...,-74.336389,3425.0,3425.0,"Bogotá, D.C., Bogotá, Localidad Sumapaz, Vered...",None,Bogotá,Bogotá,None,None,2025-06-18 09:15:47
42840,42841,NaN,Specimen,NaN,None,"Rodríguez, Ángela",2669.0,2669,11629,2019-07-16,...,-74.336389,3425.0,3425.0,"Bogotá, D.C., Bogotá, Localidad Sumapaz, Vered...",None,Bogotá,Bogotá,None,None,2025-06-18 09:15:47
42841,42842,NaN,Specimen,NaN,None,"Rodríguez, Ángela",2668.0,2668,11629,2019-07-16,...,-74.336389,3425.0,3425.0,"Bogotá, D.C., Bogotá, Localidad Sumapaz, Vered...",None,Bogotá,Bogotá,None,None,2025-06-18 09:15:47
42842,42843,NaN,Specimen,NaN,None,"Fajardo, Francisco",5598.0,5598,12363,2019-06-11,...,-74.172556,2871.0,2871.0,"Bogotá, D.C., Bogotá, Localidad de Sumapaz, Co...",None,Bogotá,Bogotá,None,None,2025-06-18 09:15:47


In [55]:
connection.close()